In [1]:
import pandas as pd
import requests
import json

In [ ]:
# Method for calling the API
def call_matrix_api(origins, destination, transit_mode=None):
    url = 'https://maps.googleapis.com/maps/api/distancematrix/json'
    key = ''  # INSERT API KEY !!!
    
    # Add transit_mode to the parameters if provided
    params = {'key': key, 'origins': origins, 'destinations': destination}
    
    if transit_mode:
        params['transit_mode'] = transit_mode

    req = requests.get(url=url, params=params)
    response = json.loads(req.content)
    return response


In [ ]:
# Read the location data
df = pd.read_excel('Book1.xlsx')

# Get the list of destinations
destinations = df['destination'].value_counts().index

In [ ]:
results = []
count_calls = 0
for destination in destinations:
    # Leave only rows with one destination
    df_temp = df[df['destination'] == destination]
    df_temp = df_temp.reset_index()

    origins = ''
    origins_array = []
    counter = 0

    # Put origins into the right format: single string with | as separator
    for location in df_temp['origin']:
        # Each string has at most 25 origins, in case of more create a list of strings
        if counter > 24:
            origins_array.append(origins)
            origins = ''
            counter = 0

        origins += location + '|'
        counter += 1

    origins_array.append(origins)

    distances = []
    times = []

    # For each string of origins, call the API with transit mode
    for origins_string in origins_array:
        api_response = call_matrix_api(origins_string, destination, transit_mode='transit')
        count_calls += 1

        # Extract transit distance and time
        if api_response['status'] == 'OK':
            for row in api_response['rows']:
                for element in row['elements']:
                    if element['status'] == 'OK':
                        distances.append(element.get('distance', {}).get('value', -1))
                        times.append(element.get('duration', {}).get('value', -1))
                    else:
                        distances.append(-1)
                        times.append(-1)

    # Create result rows and append to the results list
    for i in range(len(distances)):
        results.append([df_temp['origin'][i], destination, distances[i], times[i]])

In [ ]:
# Create a dataframe out of the results list
results_df = pd.DataFrame(results, columns=['origin', 'destination', 'transit distance [m]', 'transit time [s]'])

# Export to CSV / Excel
results_df.to_csv('transit-distances.csv')
results_df.to_excel('transit-distances.xlsx')